In [1]:
import numpy as np

In [2]:
from loadData import load_data, TRAIN_DATA_POINTS, TEST_DATA_POINTS, N_CLASSES

train_data = load_data("data/ae.train", num_data_points=TRAIN_DATA_POINTS)
test_data = load_data("data/ae.test", num_data_points=TEST_DATA_POINTS)

Maximum number of rows of all datapoints: 29
Successfully created a NumPy array with shape: (270, 29, 12)
Maximum number of rows of all datapoints: 29
Successfully created a NumPy array with shape: (370, 29, 12)


In [3]:
train_time_steps, channels = train_data.shape[1:]
test_time_steps = test_data.shape[1]

In [4]:
from processing import generate_class_matrix

class_matrix = generate_class_matrix(TRAIN_DATA_POINTS, N_CLASSES)
idxs = np.arange(TRAIN_DATA_POINTS)

In [ ]:
from DynamicTimeWarping import DTW

k_nn = 1
k_folds = 9

accuracies = []

for k_nn in range(1, 30):
    np.random.shuffle(idxs)
    shuffeled_data = train_data[idxs]
    shuffeled_classes = class_matrix[idxs]

    data_folds = np.array(np.split(shuffeled_data, k_folds))
    class_folds = np.array(np.split(shuffeled_classes, k_folds))

    misclassifications = 0
    for fold in range(k_folds):
        fold_misclassification = 0
        training_fold = np.concatenate([data_folds[idx] for idx in range(k_folds) if idx != fold])
        class_data = np.concatenate([class_folds[idx] for idx in range(k_folds) if idx != fold])

        for data_point, class_vect in zip(data_folds[fold], class_folds[fold]):
            distances = [DTW(data_point, training_fold[idx]) for idx in range(len(training_fold))]
            
            nearest_neighbors = np.zeros(N_CLASSES)
            for _ in range(k_nn):
                nearest_neighbor = np.argmin(distances)
                nearest_neighbors += class_data[nearest_neighbor]
                distances[nearest_neighbor] = np.inf

            prediction = np.argmax(nearest_neighbors)
            if prediction != np.argmax(class_vect):
                fold_misclassification += 1
                misclassifications += 1
        
        print(f"accuracy on fold {fold}:\t {100 - (fold_misclassification / data_folds.shape[1]) * 100}%")

    accuracy = 100 - (misclassifications / TRAIN_DATA_POINTS) * 100
    accuracies.append(accuracy)
    print(f"validation accuracy for k = {k_nn}:\t {accuracy}%")

accuracy on fold 0:	 86.66666666666667%
accuracy on fold 1:	 86.66666666666667%
accuracy on fold 2:	 100.0%
accuracy on fold 3:	 93.33333333333333%
accuracy on fold 4:	 86.66666666666667%
accuracy on fold 5:	 76.66666666666667%
accuracy on fold 6:	 96.66666666666667%
accuracy on fold 7:	 83.33333333333334%
accuracy on fold 8:	 90.0%
validation accuracy for k = 1:	 88.88888888888889%
accuracy on fold 0:	 83.33333333333334%
accuracy on fold 1:	 83.33333333333334%
accuracy on fold 2:	 90.0%
accuracy on fold 3:	 86.66666666666667%
accuracy on fold 4:	 86.66666666666667%
accuracy on fold 5:	 83.33333333333334%
accuracy on fold 6:	 90.0%
accuracy on fold 7:	 83.33333333333334%
accuracy on fold 8:	 83.33333333333334%
validation accuracy for k = 2:	 85.55555555555556%
accuracy on fold 0:	 93.33333333333333%
accuracy on fold 1:	 83.33333333333334%
accuracy on fold 2:	 80.0%
accuracy on fold 3:	 96.66666666666667%
accuracy on fold 4:	 96.66666666666667%
accuracy on fold 5:	 93.33333333333333%
ac

In [ ]:
accuracies

validation accuracy for k = 1:	 88.51851851851852%
